In [1]:
import pandas as pd
import os
import openai
import requests
import re

openai.api_key = ''

In [2]:
#!pip install openai


In [3]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [85]:

import requests

def get_github_folder_contents(github_folder_url):
    # Extract the owner, repository, and folder path from the given URL
    parts = github_folder_url.split('/')
    owner = parts[3]
    repo = parts[4]
    folder_path = '/'.join(parts[7:])

    # Make a request to the GitHub API to get the contents of the folder
    api_url = f'https://api.github.com/repos/{owner}/{repo}/contents/{folder_path}'
    response = requests.get(api_url)

    # Check for successful response
    if response.status_code != 200:
        raise Exception(f"Failed to fetch contents from GitHub. Status Code: {response.status_code}")

    # Parse the JSON response
    folder_contents = response.json()

    # Extract the file names from the response
    file_names = [item['name'] for item in folder_contents if item['type'] == 'file']

    return file_names

# Example usage:
github_folder_url = 'https://github.com/atreish/yukatzbot/tree/main/Text%20data/programs%20data'
file_names = get_github_folder_contents(github_folder_url)[0:]
print(file_names)


['ai.txt', 'bme.txt', 'faculty.txt', 'mathematics.txt', 'ot.txt', 'pas.txt', 'physics.txt', 'slp.txt']


In [149]:
unsuccessful=[]
sentences=[]
delimiter = "####"


# Process each text file
for file_name in file_names:
    # Download the text file content
    raw_file_url = f'https://github.com/atreish/yukatzbot/tree/main/Text%20data/programs%20data/{file_name}'
    response = requests.get(raw_file_url)
    response.raise_for_status()
    text_content = response.text

    user_message_1 = f"""
You are provided with information of all the faculty of one program with their respective websites in text_content. 
Generate questions and corresponsing answers in the following format

Who are the faculty of the artificial intelligence program?
The faculty of artificial intelligence program are Pablo Roldan, Marian Gidea, Peter Nandori and Youshan Zhang.
Who is Pablo Roldan?
Pablo Roldan is the faculty of Artificial Intelligence program in Katz school. For more details, refter to the website 'https://www.yu.edu/faculty/pages/roldan-pablo'
Marian Gidea?
Marian Gidea is the faculty of Artificial Intelligence program in Katz school. For more details, refter to the website 'https://www.yu.edu/faculty/pages/gidea-marian'
Who is Peter Nandori?
Peter Nandori is the faculty of Artificial Intelligence program in Katz school. For more details, refter to the website 'https://www.yu.edu/faculty/pages/nandori-peter'
Who is Youshan Zhang?
Youshan Zhang is the faculty of Artificial Intelligence program in Katz school. For more details, refter to the website 'https://youshanzhang.github.io/'

"""
    messages =  [
{'role':'system',
 'content': text_content},
{'role':'user',
 'content': f"{delimiter}{user_message_1}{delimiter}"},
]
    try:
        category_and_product_respons = get_completion_from_messages(messages)
        sentences.extend(category_and_product_respons.split('\n'))

    except openai.OpenAIError as e:
        # Handle the OpenAIError exception
        unsuccessful.append(text_content)
        # Additional error handling code if needed

In [150]:
len(sentences)

96

In [151]:
sentences=list(filter(lambda x: x!='I can provide five questions and their corresponding answers based on the above text:',sentences))

In [152]:
sentences=list(filter(lambda x: x!="I'm sorry, as an AI language model, I cannot provide a list of 10 questions and their corresponding answers without any context. Could you please provide more information or specify the questions you would like me to answer?",sentences))

In [153]:
sentences=list(filter(lambda x: x!='',sentences))

In [154]:
df=pd.DataFrame(columns=['question','answer'])

In [155]:
for i in range(0, len(sentences), 2):
    if i + 1 < len(sentences):
        df.loc[len(df)] = {
            'question': sentences[i],
            'answer': sentences[i + 1],
        }
    else:
        df.loc[len(df)] = {
            'question': sentences[i],
            'answer': '',
        }

In [156]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'Answer: ','',x))

In [157]:
df['question']=df['question'].apply(lambda x: re.sub(r'\d+\.\s','',x))

In [158]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'^\d+\.\s*','',x))

In [159]:
df['question']=df['question'].apply(lambda x: re.sub(r'Q:','',x))

In [160]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'Answer:','',x))

In [163]:
# Identify duplicate values in the 'question' column
duplicate_mask = df.duplicated(subset=['question'], keep='first')

# Keep only the unique values (first occurrence of each question)
df_unique = df[~duplicate_mask]

In [164]:
df_unique.to_csv('questions_answers_cleaned_faculty.csv', index=False)